In [56]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns

from PIL import Image

In [2]:
torch.manual_seed(42)
np.random.seed(42)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [32]:
batch_size = 64
num_classes = 2
learning_rate = 0.001
num_epochs = 5
classes = ["cat", "dog"]

In [62]:
def find_corrupted_images(data_dir, extensions=('.jpg', '.jpeg', '.png')):
    corrupted = []
    for img_path in Path(data_dir).rglob('*'):
        if img_path.suffix.lower() in extensions:
            try:
                img = Image.open(img_path)
                img.verify()  # Быстрая проверка целостности
            except (IOError, OSError, Image.UnidentifiedImageError) as e:
                print(f"Битый файл: {img_path} — {e}")
                corrupted.append(str(img_path))
    return corrupted

find_corrupted_images('PetImages/dog')
find_corrupted_images('PetImages/cat')

Битый файл: PetImages\dog\11702.jpg — cannot identify image file 'PetImages\\dog\\11702.jpg'
Битый файл: PetImages\cat\666.jpg — cannot identify image file 'PetImages\\cat\\666.jpg'


['PetImages\\cat\\666.jpg']

In [70]:
if os.path.exists("PetImages/dog/11702.jpg"):
    os.remove("PetImages/dog/11702.jpg")
else:
    print("Файл не найден.")

In [71]:
if os.path.exists("PetImages/cat/666.jpg"):
    os.remove("PetImages/cat/666.jpg")
else:
    print("Файл не найден.")

In [72]:
import os
import shutil
from pathlib import Path
import random

# Исходные пути
source_dir = Path("PetImages")          # здесь лежат папки cat/ и dog/
output_dir = Path("dataset")       # сюда сохраним train/ и test/

# Создаём целевые папки
for split in ["train", "test"]:
    for cls in ["cat", "dog"]:
        (output_dir / split / cls).mkdir(parents=True, exist_ok=True)

# Параметры
train_ratio = 0.8  # 80% на обучение, 20% на тест

# Обрабатываем каждый класс
for cls in ["cat", "dog"]:
    src_cls_dir = source_dir / cls
    all_files = list(src_cls_dir.glob("*.jpg"))  # или *.png, если нужно
    random.shuffle(all_files)

    n_train = int(len(all_files) * train_ratio)
    train_files = all_files[:n_train]
    test_files = all_files[n_train:]

    # Копируем файлы (можно использовать move, если не нужно сохранять оригинал)
    for f in train_files:
        shutil.copy(f, output_dir / "train" / cls / f.name)
    for f in test_files:
        shutil.copy(f, output_dir / "test" / cls / f.name)

In [ ]:
# files = sorted(os.listdir(folder_path))
# image_files = [f for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# # Сначала временно переименуем
# for i, filename in enumerate(image_files):
#     old_path = os.path.join(folder_path, filename)
#     tmp_path = os.path.join(folder_path, f"tmp_{i}.tmp")
#     os.rename(old_path, tmp_path)

# # Теперь окончательно в нужный формат
# tmp_files = sorted(os.listdir(folder_path))
# for idx, filename in enumerate(tmp_files):
#     if filename.startswith("tmp_"):
#         ext = ".jpg"  # или подставь свой формат
#         new_path = os.path.join(folder_path, f"{idx}{ext}")
#         os.rename(os.path.join(folder_path, filename), new_path)


In [73]:
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder('dataset/train', transform=transform)
test_dataset = datasets.ImageFolder('dataset/test', transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [28]:
# class LeNet5(nn.Module):
#     def __init__(self, num_classes):
#         super().__init__()
#         self.layer1 = nn.Sequential(
#         nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
#         nn.BatchNorm2d(6),
#         nn.ReLU(),
#         nn.MaxPool2d(kernel_size = 2, stride = 2))
#         self.layer2 = nn.Sequential(
#         nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
#         nn.BatchNorm2d(16),
#         nn.ReLU(),
#         nn.MaxPool2d(kernel_size = 2, stride = 2))
#         self.fc = nn.Linear(400, 120)
#         self.relu = nn.ReLU()
#         self.fc1 = nn.Linear(120, 84)
#         self.relu1 = nn.ReLU()
#         self.fc2 = nn.Linear(84, num_classes)

#     def forward(self, x):
#         out = self.layer1(x)
#         out = self.layer2(out)
#         out = out.reshape(out.size(0), -1)
#         out = self.fc(out)
#         out = self.relu(out)
#         out = self.fc1(out)
#         out = self.relu1(out)
#         out = self.fc2(out)
#         return out

In [74]:
class LeNet5_224(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # 16 * 53 * 53 = 44944
        self.fc = nn.Linear(44944, 120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = self.relu(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        return x

In [75]:
model = LeNet5_224(num_classes).to(device)

cost = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [76]:
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        images = images.to(device)
        labels = labels.to(device)
            
        #Forward pass
        outputs = model(images)
        loss = cost(outputs, labels)
        
        #Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [1/313], Loss: 0.6948
Epoch [1/5], Step [2/313], Loss: 1.5343
Epoch [1/5], Step [3/313], Loss: 8.3595
Epoch [1/5], Step [4/313], Loss: 8.7275
Epoch [1/5], Step [5/313], Loss: 2.6319
Epoch [1/5], Step [6/313], Loss: 0.7921
Epoch [1/5], Step [7/313], Loss: 1.1010
Epoch [1/5], Step [8/313], Loss: 0.7173
Epoch [1/5], Step [9/313], Loss: 0.9590
Epoch [1/5], Step [10/313], Loss: 0.7447
Epoch [1/5], Step [11/313], Loss: 0.6945
Epoch [1/5], Step [12/313], Loss: 0.9874
Epoch [1/5], Step [13/313], Loss: 0.6925
Epoch [1/5], Step [14/313], Loss: 0.7522
Epoch [1/5], Step [15/313], Loss: 0.7403
Epoch [1/5], Step [16/313], Loss: 0.7432
Epoch [1/5], Step [17/313], Loss: 0.7874
Epoch [1/5], Step [18/313], Loss: 0.6999
Epoch [1/5], Step [19/313], Loss: 0.6892
Epoch [1/5], Step [20/313], Loss: 0.7472
Epoch [1/5], Step [21/313], Loss: 0.6991
Epoch [1/5], Step [22/313], Loss: 0.6857
Epoch [1/5], Step [23/313], Loss: 0.6962
Epoch [1/5], Step [24/313], Loss: 0.6648
Epoch [1/5], Step [25/313